# Pipeline MLP (Random Search) para previsão de Geração de Energia Solar — Notebook

Este notebook implementa um pipeline completo **usando MLP (sklearn.neural_network.MLPRegressor)** e uma busca aleatória (*random search*) de hiperparâmetros. Ele inclui explicações linha-a-linha sobre variáveis, funções, parâmetros e decisões. Salve o arquivo `solar france.xlsx` no mesmo diretório antes de executar as células.

## 1) Instruções iniciais

- Rode o notebook célula-a-célula.
- Se executar no Colab, instale dependências se necessário (por padrão `scikit-learn`, `pandas`, `matplotlib` já vêm no Colab). Se não tiver, execute: `!pip install -q scikit-learn pandas matplotlib openpyxl joblib`.
- O notebook fará detecção automática de `target` e `date` e gerará lags se não houver preditores adicionais.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import joblib
from pathlib import Path
import random, time, os

## 2) Carregar dados

Leia o arquivo Excel `solar france.xlsx`.

In [ ]:
CAMINHO_RAIZ = Path(os.getcwd()).resolve().parent.parent
OUTPUT = CAMINHO_RAIZ / 'out_geracao_energia'
os.makedirs(OUTPUT, exist_ok=True)

solar = pd.read_excel(CAMINHO_RAIZ / 'data' / 'solar france.xlsx')
print('Dimensões do dataset:', solar.shape)
print('Colunas:', solar.columns.tolist())
display(solar.head(8))

## 3) Transformar série em problema supervisionado (gerar lags)

Se não houver preditores além de `date` e `target`, geramos *lags* (defasagens) para criar features a partir da própria série temporal. Aqui criamos 8 lags por padrão (pode ajustar). Cada nova feature `lag_k` representa o valor do target há k períodos atrás.

In [ ]:
target_col = 'Production'
date_col = 'Date and Hour'
# criar lags automáticos
n_lags = 8
if date_col is not None:
    solar = solar.sort_values(by=date_col).reset_index(drop=True)
for lag in range(1, n_lags+1):
    solar[f'lag_{lag}'] = solar[target_col].shift(lag)
solar = solar.dropna().reset_index(drop=True)
feature_cols = [f'lag_{lag}' for lag in range(1, n_lags+1)]
print('Nenhum preditor encontrado além do target/date. Criadas features:', feature_cols)
# Exibir amostra
display(solar.head(8))

## 4) Separar X (features) e y (target) e criar split (treino/validação/teste)

usamos split cronológico: 50% treino, 25% validação, 25% teste.

In [ ]:
# Definir X e y
X = solar[feature_cols].copy()
y = solar[target_col].copy()
print('Shapes: X=', X.shape, ' y=', y.shape)

# Split cronológico
df_sorted = solar.sort_values(by=date_col).reset_index(drop=True)
X_sorted = df_sorted[feature_cols]
y_sorted = df_sorted[target_col]
n = len(df_sorted)
n_train = int(n * 0.50)
n_val = int(n * 0.25)
X_train = X_sorted.iloc[:n_train].reset_index(drop=True)
y_train = y_sorted.iloc[:n_train].reset_index(drop=True)
X_val = X_sorted.iloc[n_train:n_train+n_val].reset_index(drop=True)
y_val = y_sorted.iloc[n_train:n_train+n_val].reset_index(drop=True)
X_test = X_sorted.iloc[n_train+n_val:].reset_index(drop=True)
y_test = y_sorted.iloc[n_train+n_val:].reset_index(drop=True)
print(f'Split cronológico: train={len(X_train)}, val={len(X_val)}, test={len(X_test)}')

## 6) Pré-processamento

- **Imputação**: SimpleImputer(strategy='median') para colunas numéricas (robusto a outliers).
- **Escalonamento**: StandardScaler — redes MLP convergem melhor com features padronizadas.
- **OneHotEncoder** para categóricas, se houver.

Usamos `ColumnTransformer` para aplicar as transformações apenas aos tipos correspondentes.

In [ ]:
# Detectar tipos
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X.select_dtypes(include=['object','category','bool']).columns.tolist()

print('numeric_cols:', numeric_cols)
print('categorical_cols:', categorical_cols)

num_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                  ('scaler', StandardScaler())])
cat_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                                  ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[('num', num_transformer, numeric_cols),
                                               ('cat', cat_transformer, categorical_cols)],
                                 remainder='drop')

# Ajustar apenas no treino (evita vazamento)
preprocessor.fit(X_train)
X_train_proc = preprocessor.transform(X_train)
X_val_proc = preprocessor.transform(X_val)
X_test_proc = preprocessor.transform(X_test)

print('Formas após pré-processamento:',
      'X_train_proc=', X_train_proc.shape,
      'X_val_proc=', X_val_proc.shape,
      'X_test_proc=', X_test_proc.shape)

## 7) Random Search (busca aleatória) para MLPRegressor

Explicações dos hiperparâmetros que vamos variar:
- `hidden_layer_sizes`: tupla com número de neurônios por camada (ex: (64, 32) → 2 camadas ocultas)
- `activation`: função de ativação (relu, tanh)
- `alpha`: termo de regularização L2 (evita overfitting)
- `learning_rate_init`: taxa inicial de aprendizado
- `solver`: algoritmo de otimização ('adam' recomendado)
- `batch_size`: tamanho do batch

Estratégia: amostrar aleatoriamente N combinações do espaço e treinar cada modelo no conjunto de treino, avaliando no conjunto de validação. Selecionamos o modelo com menor MSE na validação.

In [ ]:
# Espaço de busca
param_grid = {
    'hidden_layer_sizes': [(16,), (32,), (64,), (64,32), (128,64)],
    'activation': ['relu', 'tanh'],
    'alpha': [1e-4, 1e-3, 1e-2],
    'learning_rate_init': [1e-2, 1e-3, 1e-4],
    'solver': ['adam'],  # restringimos a 'adam' por estabilidade
    'batch_size': [16, 32, 64]
}

def sample_params(grid):
    return {k: random.choice(v) for k, v in grid.items()}

n_iter = 24  # número de combinações aleatórias a testar (ajuste conforme poder de processamento)
results = []
best = None
best_val_mse = float('inf')

start_time = time.time()
for i in range(n_iter):
    params = sample_params(param_grid)
    print(f'Iter {i+1}/{n_iter} -> {params}')
    mlp = MLPRegressor(hidden_layer_sizes=params['hidden_layer_sizes'],
                       activation=params['activation'],
                       alpha=params['alpha'],
                       learning_rate_init=params['learning_rate_init'],
                       solver=params['solver'],
                       batch_size=params['batch_size'],
                       max_iter=500,
                       early_stopping=True,
                       n_iter_no_change=20,
                       tol=1e-5,
                       random_state=42,
                       verbose=False)
    try:
        mlp.fit(X_train_proc, y_train.values.ravel())
    except Exception as e:
        print('  Treino falhou para esses hiperparâmetros:', e)
        continue
    # Avaliar no conjunto de validação
    y_val_pred = mlp.predict(X_val_proc)
    val_mse = mean_squared_error(y_val, y_val_pred)
    # MAPE poderá ser calculado depois para robustez
    print(f'  Val MSE = {val_mse:.4f}')
    results.append({'params': params, 'val_mse': val_mse, 'model': mlp})
    if val_mse < best_val_mse:
        best_val_mse = val_mse
        best = results[-1]
end_time = time.time()
print('\nRandom search concluído em {:.1f} s'.format(end_time - start_time))

## 8) Seleção do melhor modelo e avaliação final (treino/val/test)

Calculamos MSE e MAPE para treino, validação e teste no melhor modelo encontrado.

In [ ]:
# Selecionar melhor
if best is None:
    raise RuntimeError('Nenhum modelo treinado com sucesso.')

best_params = best['params']
best_model = best['model']
print('Melhor hiperparâmetros encontrados:', best_params)
print('Val MSE do melhor:', best['val_mse'])

# Função segura de MAPE (evita divisão por zero)
def safe_mape(y_true, y_pred):
    y_true = np.array(y_true).astype(float)
    y_pred = np.array(y_pred).astype(float)
    eps = 1e-8
    denom = np.where(np.abs(y_true) < eps, eps, y_true)
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100.0

# Previsões
y_train_pred = best_model.predict(X_train_proc)
y_val_pred = best_model.predict(X_val_proc)
y_test_pred = best_model.predict(X_test_proc)

# Métricas
mse_train = mean_squared_error(y_train, y_train_pred)
mape_train = safe_mape(y_train, y_train_pred)
mse_val = mean_squared_error(y_val, y_val_pred)
mape_val = safe_mape(y_val, y_val_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mape_test = safe_mape(y_test, y_test_pred)

print('\nMétricas finais:')
print(f'Train - MSE: {mse_train:.4f}, MAPE: {mape_train:.4f}%')
print(f'Val   - MSE: {mse_val:.4f}, MAPE: {mape_val:.4f}%')
print(f'Test  - MSE: {mse_test:.4f}, MAPE: {mape_test:.4f}%')

## 9) Gráficos: reais vs previstos e histogramas de resíduos

Mostramos gráficos para treino, validação e teste.

In [ ]:
# Gráficos (um por célula)
def plot_actual_vs_pred(y_true, y_pred, title):
    plt.figure(figsize=(7,4))
    plt.scatter(y_true, y_pred)
    plt.xlabel('Valor real')
    plt.ylabel('Valor previsto')
    plt.title(title)
    mn = min(np.min(y_true), np.min(y_pred))
    mx = max(np.max(y_true), np.max(y_pred))
    plt.plot([mn,mx],[mn,mx], linewidth=1)
    plt.savefig(OUTPUT/ f'MLP - {title}.png')
    plt.show()

def plot_residuals_hist(y_true, y_pred, title):
    residuals = y_true - y_pred
    plt.figure(figsize=(7,4))
    plt.hist(residuals, bins=30)
    plt.xlabel('Resíduo')
    plt.ylabel('Frequência')
    plt.title(title)
    plt.savefig(OUTPUT/ f'MLP - {title}.png')
    plt.show()

# Treino
plot_actual_vs_pred(y_train, y_train_pred, 'Treino - Reais vs Previstas')
plot_residuals_hist(y_train, y_train_pred, 'Treino - Resíduos')

# Validação
plot_actual_vs_pred(y_val, y_val_pred, 'Validação - Reais vs Previstas')
plot_residuals_hist(y_val, y_val_pred, 'Validação - Resíduos')

# Teste
plot_actual_vs_pred(y_test, y_test_pred, 'Teste - Reais vs Previstas')
plot_residuals_hist(y_test, y_test_pred, 'Teste - Resíduos')

In [ ]:
datasets = ['Treinamento', 'Validação', 'Teste']
mse_values = [mse_train, mse_val, mse_test]
mape_values = [mape_train, mape_val, mape_test]

# Criar figura com 2 subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# --- Gráfico 1: MSE ---
axes[0].bar(datasets, mse_values, color=['#4CAF50', '#FFC107', '#2196F3'])
axes[0].set_title('Comparação do MSE (Erro Quadrático Médio)')
axes[0].set_ylabel('MSE')
axes[0].grid(True, axis='y', linestyle='--', alpha=0.6)

# --- Gráfico 2: MAPE ---
axes[1].bar(datasets, mape_values, color=['#4CAF50', '#FFC107', '#2196F3'])
axes[1].set_title('Comparação do MAPE (Erro Percentual Absoluto Médio)')
axes[1].set_ylabel('MAPE (%)')
axes[1].grid(True, axis='y', linestyle='--', alpha=0.6)

plt.suptitle('Comparação de desempenho do modelo MLP', fontsize=14)
plt.tight_layout()
plt.savefig(OUTPUT / 'MLP - Comparação de desempenho do modelo.png')
plt.show()